In [261]:
# Import necessary packages
import geopandas as gpd
import osmnx as ox

place_name = "Youssoufia, Morocco"
G = ox.graph_from_place(place_name, network_type='all_private')

gdf = gpd.read_file('map.geojson')

In [262]:
# Convert the graph to GeoDataFrames
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

In [263]:
gdf_nodes.drop(gdf_nodes[gdf_nodes['highway'].isnull()].index, inplace=True)

In [264]:
gdf_nodes['geometry'][1109007239].coords

In [10]:
# Find popup slice
def find_popup_slice(html):
    '''
    Find the strating and ending index of popup function
    '''

    pattern = 'function latLngPop(e)'

    # Starting index
    starting_index = html.find(pattern)

    # Ending index
    tmp_html = html[starting_index:]

    # Look over the tmp html
    found = 0
    index = 0
    openning_found = False
    while not openning_found or found > 0:
        if tmp_html[index] == '{':
            found += 1
            openning_found = True
        elif tmp_html[index] == '}':
            found -= 1

        index += 1

    # Determine the ending index
    ending_index = starting_index + index

    return starting_index, ending_index

In [11]:
# Find map name
def find_map_variable_name(html):
    pattern = 'var map_'

    starting_index = html.find(pattern) + 4
    tmp_html = html[starting_index:]
    ending_index = tmp_html.find(' =') + starting_index

    return html[starting_index:ending_index]

# Find popup name
def find_popup_variable_name(html):
    pattern = 'var lat_lng'

    starting_index = html.find(pattern) + 4
    tmp_html = html[starting_index:]
    ending_index = tmp_html.find(' =') + starting_index

    return html[starting_index:ending_index]

In [265]:
# Define custom code function
def custom_code(popup_variable_name, map_variable_name, folium_port):
    return '''
        // Custom code
        var coords = []
        function calculateDistance(coords) {
            if (coords.length === 2) {
                var lat1 = coords[0][0];
                var lon1 = coords[0][1];
                var lat2 = coords[1][0];
                var lon2 = coords[1][1];

                // Convert latitude and longitude from degrees to radians
                var radLat1 = lat1 * Math.PI / 180;
                var radLon1 = lon1 * Math.PI / 180;
                var radLat2 = lat2 * Math.PI / 180;
                var radLon2 = lon2 * Math.PI / 180;

                // Haversine formula to calculate distance
                var dLat = radLat2 - radLat1;
                var dLon = radLon2 - radLon1;
                var a = Math.sin(dLat / 2) * Math.sin(dLat / 2) +
                        Math.cos(radLat1) * Math.cos(radLat2) *
                        Math.sin(dLon / 2) * Math.sin(dLon / 2);
                var c = 2 * Math.atan2(Math.sqrt(a), Math.sqrt(1 - a));
                var distance = 6371000 * c; // Earth's radius in meters

                return (distance < 1000) ? 'La distance est de '+distance.toFixed(2)+' mètres' : 'La distance est de '+(distance/ 1000).toFixed(2)+' km';
            }
            return null;
        }
        function latLngPop(e) {
            %s
                .setLatLng(e.latlng)
                .setContent(`
                    ${coords.length === 1 ? "Calculer la distance" : `lat: ${e.latlng.lat}, lng: ${e.latlng.lng}`}
                    <button onClick="
                        fetch('http://localhost:%s', {
                            method: 'POST',
                            mode: 'no-cors',
                            headers: {
                                'Accept': 'application/json',
                                'Content-Type': 'application/json'
                            },
                            body: JSON.stringify({
                                latitude: ${e.latlng.lat},
                                longitude: ${e.latlng.lng}
                            })
                        });

                        L.marker(
                            [${e.latlng.lat}, ${e.latlng.lng}],
                            {}
                        ).addTo(%s);

                        coords.push([${e.latlng.lat}, ${e.latlng.lng}]);

                        if (coords.length == 2) {
                            var distance = calculateDistance(coords)
                            if (distance !== null) {
                                console.log(distance)
                                L.popup()
                                    .setLatLng([(coords[0][0] + coords[1][0]) / 2, (coords[0][1] + coords[1][1]) / 2])
                                    .setContent(distance)
                                    .openOn(%s);
                                    L.polyline(coords).addTo(%s);
                            }
                            coords = [];
                        }
                    "> 
                    Save coordinate
                    </button>
                    <button onClick="
                        fetch('http://localhost:%s', {
                            method: 'POST',
                            mode: 'no-cors',
                            headers: {
                                'Accept': 'application/json',
                                'Content-Type': 'application/json'
                            },
                            body: 'q'
                        });
                    ">
                        Quit
                    </button>
                `).openOn(%s);
        }
        // End custom code
    '''% (popup_variable_name, 
          folium_port, 
          map_variable_name,
          map_variable_name, 
          map_variable_name, 
          folium_port, 
          map_variable_name
        )

In [266]:
# Create folium map

import folium 
def create_folium_map(map_file, center_coord, folium_port):
    
    # Create a folium map
    map_youssoufia = folium.Map(location=center_coord, zoom_start=13)

    # Ajouter les nœuds sur la carte
    for idx, node in gdf_nodes.iterrows():
        marker = folium.Marker(
            [node.geometry.y, node.geometry.x], 
            popup=node['highway']
        )
        marker.add_to(map_youssoufia)

    for x, y in gdf.iterrows():
        if x >= 2:
            color = y['marker-color']
            folium.Marker(location=[y.geometry.y, y.geometry.x], popup=y['name'], icon=folium.Icon(color=color)).add_to(map_youssoufia)
        else :
            folium.PolyLine(locations=y.geometry.coords, color='blue').add_to(map_youssoufia)

    # Ajouter les arêtes sur la carte
    for idx, edge in gdf_edges.iterrows():
        folium.PolyLine(locations=edge.geometry.coords, color='green').add_to(map_youssoufia)

    # Add popup
    folium.LatLngPopup().add_to(map_youssoufia)

    # Save the map to html
    map_youssoufia.save(map_file)

    # Read the folium file
    html = None
    with open(map_file, 'r') as mapfile:
        html = mapfile.read()

    # Determine popup function indecises
    pstart, pend = find_popup_slice(html) 

    # Find map variable name
    map_variable_name = find_map_variable_name(html)

    # Find map variable name
    popup_variable_name = find_popup_variable_name(html)

    # Inject code
    with open(map_file, 'w') as mapfile:
        mapfile.write(
            html[:pstart] +\
            custom_code(popup_variable_name, map_variable_name, folium_port)+\
            html[pend:]
        )

In [14]:
# Import selenium packages
from selenium import webdriver
# Open the folium map selenium
def open_folium_map(project_url, map_file):
    driver = None
    try:
        driver = webdriver.Firefox()
        driver.get(
            project_url = map_file
        )
    except Exception as ex:
        print('Driver failed ton open/find url: ', ex)

    return driver

In [267]:
# Calculate distance

# Import geopy distance
from geopy.distance import geodesic

def calculate_distance(coords):
    if type(coords[0]) == str:
        coords = [eval(x) for x in coords]

    first_point = (coords[0]['latitude'], coords[0]['longitude'])
    second_point = (coords[1]['latitude'], coords[1]['longitude'])

    distance = geodesic(first_point, second_point)
    if distance.km < 1:
        print(f'La distance entre les deux points est de {round(distance.m, 2)} mètres')
    else:
        print(f'La distance entre les deux points est de {round(distance.km, 2)} km')

In [269]:
# Import base http request
from http.server import BaseHTTPRequestHandler, HTTPServer

class FoliumServer(BaseHTTPRequestHandler):
    coords = []
    def _set_response(self):
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()

    def do_POST(self):
        content_length = int(self.headers['Content-Length'])
        post_data = self.rfile.read(content_length)

        data = post_data.decode('utf-8')

        if data.lower() == 'q':
            raise KeyboardInterrupt('Exit webserver')
        self.coords.append((data))


        if len(self.coords) == 2:
            tmp_coords = self.coords
            self.coords = []
            calculate_distance(tmp_coords)

        self._set_response()

# Listen to folium  map
def listen_to_folium_map(port=3001):
    server_address = ('', port)
    httpd = HTTPServer(server_address, FoliumServer)
    print('Server started')

    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        pass
    except Exception:
        pass

    httpd.server_close()
    print('Server stopped')

In [270]:
# Create variables
folium_port = 3001
center_coord = [32.23630130013652, -8.51364643783117]
map_file = 'map_youssoufia.html'
coords = []

# Create the folium map
create_folium_map(map_file, center_coord, folium_port)

# Open the folium map selenium
# project_map_url = 'file:///home/like/Documents/Projects/Youcode/Challenges/city_map_points/'
# driver = open_folium_map(project_map_url, map_file)

# Run webserver that listen to sent coordinates
listen_to_folium_map(port=folium_port)
# Close the folium map

# Print all collected coords

/tmp/ipykernel_40185/717955007.py:20: UserWarning: color argument of Icon should be one of: {'blue', 'white', 'pink', 'purple', 'red', 'lightgray', 'darkblue', 'green', 'darkgreen', 'lightgreen', 'orange', 'lightred', 'gray', 'lightblue', 'beige', 'darkpurple', 'darkred', 'cadetblue', 'black'}.
  folium.Marker(location=[y.geometry.y, y.geometry.x], popup=y['name'], icon=folium.Icon(color=color)).add_to(map_youssoufia)


Server started


127.0.0.1 - - [01/Aug/2023 23:24:44] "POST / HTTP/1.1" 200 -


La distance entre les deux points est de 628.2 mètres


127.0.0.1 - - [01/Aug/2023 23:24:49] "POST / HTTP/1.1" 200 -


Server stopped
